In [57]:
import json
import pandas as pd

# 将爬取到的数据整理到.csv文件中
with open('shanghai.jsonl', 'r', encoding='utf-8') as f:
    lines = f.readlines()

data = []
for line in lines:
    data.append(json.loads(line))

df = pd.DataFrame(data)

# 根据jobId进行去重
df.drop_duplicates(subset='jobId', keep='first', inplace=True)

df.to_csv('original_output.csv', index=False)

df.head(200)
# 输出数据的数量
print(len(df.index))

6550


In [43]:
df = pd.read_csv('original_output.csv')

# 筛选出需要的特征
features = ['jobType', 'workAreaCode', 'workYear', 'degreeString', 'industryType1', 'companyTypeString', 'companySizeCode', 'jobSalaryMax', 'jobSalaryMin', 'term']
df = df[features]

df.head(200)


,jobType,workAreaCode,workYear,degreeString,industryType1,companyTypeString,companySizeCode,jobSalaryMax,jobSalaryMin,term
0,0,10000,10.0,大专,4.0,民营,1.0,3000.0,2000.0,0
1,0,11100,3.0,本科,1.0,民营,1.0,3000.0,2000.0,0
2,0,999999,3.0,本科,23.0,创业公司,1.0,3000.0,2000.0,1
3,0,10000,10.0,本科,32.0,民营,7.0,3000.0,2000.0,0
4,0,10000,10.0,NaN,31.0,民营,1.0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
195,0,10000,1.0,本科,31.0,国企,NaN,NaN,NaN,4
196,0,10000,1.0,本科,7.0,外资（非欧美）,NaN,NaN,NaN,4
197,0,10000,1.0,硕士,31.0,国企,NaN,NaN,NaN,4
198,0,10000,1.0,硕士,39.0,国企,NaN,NaN,NaN,0


In [44]:
# 检查每一列是否包含缺省值
has_null_values = df.isnull().any()
print(has_null_values)

jobType              False
workAreaCode         False
workYear              True
degreeString          True
industryType1         True
companyTypeString     True
companySizeCode       True
jobSalaryMax          True
jobSalaryMin          True
term                 False
dtype: bool


In [45]:
# 展示薪资数据的分布
step = 1000
min_salary_max = df['jobSalaryMax'].min()
max_salary_max = df['jobSalaryMax'].max()

# 使用 cut 函数将数据划分为区间，并统计数量
bins = range(int(min_salary_max), int(max_salary_max) + step, step)
data = pd.cut(df['jobSalaryMax'], bins=bins, include_lowest=True)

# 统计每个范围内的数量
salary_max_counts = data.value_counts().sort_index()

# 输出每个范围对应的数量
print(f'jobSalaryMax 变量的值范围统计，步长为 {step}:')
for range_value, count in salary_max_counts.items():
    print(f'{range_value}: {count}')

jobSalaryMax 变量的值范围统计，步长为 1000:
(1000.999, 2001.0]: 79
(2001.0, 3001.0]: 127
(3001.0, 4001.0]: 252
(4001.0, 5001.0]: 518
(5001.0, 6001.0]: 428
(6001.0, 7001.0]: 172
(7001.0, 8001.0]: 432
(8001.0, 9001.0]: 96
(9001.0, 10001.0]: 295
(10001.0, 11001.0]: 22
(11001.0, 12001.0]: 219
(12001.0, 13001.0]: 45
(13001.0, 14001.0]: 30
(14001.0, 15001.0]: 256
(15001.0, 16001.0]: 64
(16001.0, 17001.0]: 14
(17001.0, 18001.0]: 43
(18001.0, 19001.0]: 2
(19001.0, 20001.0]: 230
(20001.0, 21001.0]: 3
(21001.0, 22001.0]: 10
(22001.0, 23001.0]: 5
(23001.0, 24001.0]: 11
(24001.0, 25001.0]: 131
(25001.0, 26001.0]: 5
(26001.0, 27001.0]: 1
(27001.0, 28001.0]: 9
(28001.0, 29001.0]: 0
(29001.0, 30001.0]: 290
(30001.0, 31001.0]: 0
(31001.0, 32001.0]: 2
(32001.0, 33001.0]: 1
(33001.0, 34001.0]: 0
(34001.0, 35001.0]: 101
(35001.0, 36001.0]: 5
(36001.0, 37001.0]: 0
(37001.0, 38001.0]: 0
(38001.0, 39001.0]: 0
(39001.0, 40001.0]: 333
(40001.0, 41001.0]: 0
(41001.0, 42001.0]: 0
(42001.0, 43001.0]: 0
(43001.0, 44001.0]: 1

In [46]:
# 缺省值处理
df.dropna(subset=['jobSalaryMax', 'jobSalaryMin'], inplace=True)
df['workYear'] = df['workYear'].fillna(0)
df['workYear'] = df['workYear'].replace(10, 0)
df['industryType1'] = df['industryType1'].fillna(0)
df['degreeString'] = df['degreeString'].fillna('None')
df['companyTypeString'] = df['companyTypeString'].fillna('None')
median_company_size = df['companySizeCode'].median()
df['companySizeCode'] = df['companySizeCode'].fillna(median_company_size)

In [47]:
# 类型转换
degree_mapping = {
    'None': 0,
    '初中及以下': 1,
    '中技/中专': 2,
    '高中': 3,
    '大专': 4,
    '本科': 5,
    '硕士': 6,
    '博士': 7,
}
df['degreeString'] = df['degreeString'].map(degree_mapping)
df.rename(columns={'degreeString': 'degree'}, inplace=True)
type_mapping = {
    'None': 0,
    '民营': 1,
    '外资（欧美）': 2,
    '外资（非欧美）': 3,
    '合资': 4,
    '国企': 5,
    '已上市': 6,
    '非营利组织': 7,
    '创业公司': 8,
    '事业单位': 9,
    '外企代表处': 10,
    '政府机关': 11
}
df['companyTypeString'] = df['companyTypeString'].map(type_mapping)
df.rename(columns={'companyTypeString': 'companyType'}, inplace=True)

In [48]:
def salary_level(salary):
    if salary < 8000:
        return 1
    elif 8000 <= salary < 20000:
        return 2
    else:
        return 3

df['jobSalaryMaxLevel'] = df['jobSalaryMax'].apply(salary_level)
df['jobSalaryMinLevel'] = df['jobSalaryMin'].apply(salary_level)

df.to_csv('new_output.csv', index=False)

df.head(100)

,jobType,workAreaCode,workYear,degree,industryType1,companyType,companySizeCode,jobSalaryMax,jobSalaryMin,term,jobSalaryMaxLevel,jobSalaryMinLevel
0,0,10000,0.0,4,4.0,1,1.0,3000.0,2000.0,0,1,1
1,0,11100,3.0,5,1.0,1,1.0,3000.0,2000.0,0,1,1
2,0,999999,3.0,5,23.0,8,1.0,3000.0,2000.0,1,1,1
3,0,10000,0.0,5,32.0,1,7.0,3000.0,2000.0,0,1,1
5,0,10500,0.0,0,60.0,2,5.0,3000.0,2000.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
146,0,10600,1.0,4,48.0,5,1.0,3000.0,2000.0,0,1,1
147,0,10600,1.0,5,61.0,4,5.0,4000.0,2000.0,0,1,1
148,0,100202,1.0,5,48.0,1,1.0,3000.0,2000.0,0,1,1
149,0,10000,1.0,6,3.0,5,5.0,3000.0,1500.0,0,1,1


In [49]:
has_null_values = df.isnull().any()
print(has_null_values)

jobType              False
workAreaCode         False
workYear             False
degree               False
industryType1        False
companyType          False
companySizeCode      False
jobSalaryMax         False
jobSalaryMin         False
term                 False
jobSalaryMaxLevel    False
jobSalaryMinLevel    False
dtype: bool


In [52]:
# 合并
df1 = pd.read_csv('new_output.csv')
df2 = pd.read_csv('SH_new_output.csv')

merged_df = pd.concat([df1, df2])

merged_df.to_csv('output.csv', index=False)